In [ ]:
# default_exp main

# rfpy - CFRS Rfeye Node Logger binaries parser

In [ ]:
#hide
%load_ext autoreload
%autoreload 2            #Reload the code automatically

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
from typing import *
import os
from fastcore.xtras import Path
from fastcore.script import *
from fastcore.utils import parallel, partial
from rfpy.parser import *
import pandas as pd
from rfpy.utils import *
from rich.progress import track, Progress

In [ ]:
#export
def read_meta(filename):
    ext = filename.suffix
    if ext == '.csv':
        df = pd.read_csv(filename)
    elif ext == '.xlsx':
        df = pd.read_excel(filename, engine='openpyxl')
    elif ext == '.fth':
        df = pd.read_feather(filename)
        if "wallclock_datetime" in df.columns:
            df.set_index('wallclock_datetime', inplace=True)
    else:
        raise ValueError(f"Extension {ext} not implemented")
    return df
    
@call_parse
def process_bin(entrada: Param("Diretório contendo arquivos .bin", str),
                saida: Param("Diretório para salvar os arquivos de saída", str),
                recursivo: Param("Buscar arquivos de maneira recursiva?", store_true)=False,
                pastas: Param("Limita a busca às pastas", Iterable[str])=None,
                meta: Param("Extrair e Salvar os metadados?", store_true)=False,
                levels: Param("Extrair e Salvar os níveis de Espectro?", store_true)=False,
                ext: Param("Qual extensão salvar os arquivos", str)='.fth', 
                substituir: Param("Lista de arquivos já processados", store_true)=False):

        lista_bins = get_files(entrada, extensions=['.bin'], recurse=recursivo, folders=pastas)  
        parsed_bins = {}
        meta_path = Path(f'{saida}/meta')
        levels_path = Path(f'{saida}/levels')
        meta_path.mkdir(exist_ok=True, parents=True)
        levels_path.mkdir(exist_ok=True, parents=True)

        processed = Path(f'{saida}/processed_log.txt')
        if substituir:
            done = set()
        elif processed.exists():
            done = set(processed.read_text().split('\n'))
        else:
            done = set()

        console.rule("Lista de Arquivos a serem processados", style='bold red')
        console.print(lista_bins, style = "bold white", overflow='fold', justify='center')
        if not lista_bins:
            return
        
        erro_formato = False
        erro_novo = False

        with Progress() as progress:
            bins = progress.track(lista_bins, total=len(lista_bins), description="[green]Processando Arquivos Binários")
            for file in bins:
                progress.description = f'[cyan]Mapeando Blocos do arquivo {file.name}'
                parsed_bins[file.stem] = parse_bin(file, progress=progress)
            if meta:
                lista = [f for f in lista_bins if f.name not in done]
                if not lista:
                    erro_novo = True
                else:
                    bins = progress.track(lista, total=len(lista), description="[green]Exportando Metadados")
                    for file in bins:
                        export_meta(file.stem, parsed_bins[file.stem], meta_path, ext=ext)
                        done.add(file.name)
            if levels:
                if ext == '.csv':
                    erro_formato = True
                    lista = []
                else:
                    lista = [f for f in lista_bins if f.name not in done]
                    if not lista:
                        erro_novo = True
                    else:
                        bins = progress.track(lista, total=len(lista), description="[green]Exportando Dados de Espectro")
                        for file in bins:
                            meta_files = get_files(meta_path)
                            for f in meta_files:
                                if file.stem in str(f):
                                    meta_file = f
                                    break
                            else:
                                console.print('[red] :poop: Não foram encontrados os arquivos com os metadados, eles serão extraídos primeiro')
                                export_meta(file.stem, parsed_bins[file.stem], meta_path, ext=ext)
                                done.add(file.name)
                            meta_df = read_meta(meta_file)
                            index=meta_df.index
                            export_level(file.stem, parsed_bins[file.stem], levels_path, ext=ext, index=index)
                            done.add(file.name)
        if erro_formato:
            console.print(":warning: Dados espectrais em .csv não são suportados pela explosão no tamanho de arquivo :exclamation:")
            console.print("Sugerimos o formato:point_right: [red].fth")  
        elif erro_novo:
            console.print(":sleeping: Nenhum arquivo novo a processar :zzz:")
            console.print(":point_up: use --substituir no terminal ou substituir=True na chamada caso queira reprocessar os bins e sobrepôr os arquivos existentes :wink:")
        else:       
            processed.write_text('\n'.join(sorted(list(done))))
            console.print("kbô :satisfied:")

In [ ]:
# entrada = Path(r'C:\Users\rsilva\Downloads\entrada\pmec')
# saida = Path(r'C:\Users\rsilva\Downloads\saida')
# process_bin(entrada, saida, ext='.xlsx', levels=True)

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
Converted index.ipynb.
